### Data streams
Define interface to loose-couple utility

In [1]:
interface Writable {
    
    void write(DataOutputStream out) throws IOException;
    void read(DataInputStream in) throws IOException;
    
}

Utility methods

In [2]:
import java.lang.reflect.Array;
import java.io.File;

<W extends Writable> void write(File dataFile, W... objs) throws IOException {
    try(DataOutputStream out = new DataOutputStream(new BufferedOutputStream(new FileOutputStream(dataFile)))) {
        out.writeInt(objs.length);
        for (W o: objs) {
            o.write(out);
        }
    }
}

In [3]:
<W extends Writable> W[] read(Class<W> clazz, File dataFile) throws IOException, InstantiationException, IllegalAccessException {
    try(DataInputStream in = new DataInputStream(new BufferedInputStream(new FileInputStream(dataFile)))) {
        int len = in.readInt();
        W[] data = (W[]) Array.newInstance(clazz, len);
        for (int i = 0; i < len; i++) {
            data[i] = (W) clazz.newInstance();
            data[i].read(in);
        }
        return data;
    }
}

Example class

In [4]:
class Product implements Writable {
    
    private String name;
    private double unitPrice;
    private int stock;
    
    Product() {}

    Product(String name, double unitPrice, int stock) {
        this.name = name;
        this.unitPrice = unitPrice;
        this.stock = stock;
    }

    @Override
    public void write(DataOutputStream out) throws IOException {
        out.writeUTF(name);
        out.writeDouble(unitPrice);
        out.writeInt(stock);
    }

    @Override
    public void read(DataInputStream in) throws IOException {
        name = in.readUTF();
        unitPrice = in.readDouble();
        stock = in.readInt();
    }
    
    public String toString() {
        return String.format("Product{name=%s,unitPrice=%f,stock=%d}", name, unitPrice, stock);
    }

}

Create a few objects

In [5]:
Product[] store = new Product[] {
    new Product("Pienas", 1.20, 1000),
    new Product("Sviestas", 1.75, 1000)
};

In [6]:
for (Product p: store) System.out.println(p);

Product{name=Pienas,unitPrice=1.200000,stock=1000}
Product{name=Sviestas,unitPrice=1.750000,stock=1000}


Write "store" to file

In [7]:
File f = new File("store.dat");
write(f, store);

Reading objects from file

In [8]:
Product[] store1 = read(Product.class, f);

In [9]:
for (Product p: store1) System.out.println(p);

Product{name=Pienas,unitPrice=1.200000,stock=1000}
Product{name=Sviestas,unitPrice=1.750000,stock=1000}


### Object streams

To/from streams

In [10]:
@SuppressWarnings("unchecked")
<T> T read(InputStream in) throws IOException, ClassNotFoundException {
    try (ObjectInputStream oos = new ObjectInputStream(in)) {
        return (T) oos.readObject();
    }
}

<T> void write(OutputStream os, T o) throws IOException {
    try (ObjectOutputStream oos = new ObjectOutputStream(os)) {
        oos.writeObject(o);
    }
}

To/from bytes

In [11]:
<T> T readFromBytes(byte[] data) throws IOException, ClassNotFoundException {
    try (ByteArrayInputStream bin = new ByteArrayInputStream(data)) {
        return (T) read(bin);
    }
}

<T> byte[] writeToBytes(T o) throws IOException {
    try (ByteArrayOutputStream bos = new ByteArrayOutputStream()) {
        write(bos, o);
        return bos.toByteArray();
    }
}

To/from files

In [12]:
<T> void writeToFile(String fileName, T o) throws IOException {
    try (FileOutputStream fos = new FileOutputStream(fileName)) {
        write(fos, o);
    }
}

<T> T readFromFile(String fileName) throws IOException, ClassNotFoundException {
    try (FileInputStream fin = new FileInputStream(fileName)) {
        return (T) read(fin);
    }
}

Define some classes

In [13]:
enum Color {
    RED,
    BLUE,
    GREEN,
    BEIGE,
    WHITE
}

enum Fuel {
    PETROL,
    DIESEL,
    ELECTRICITY
}

In [14]:
class Engine {

    //private static final long serialVersionUID = 1L;

    private Fuel fuel;
    private float volume;

    public Engine(Fuel fuel, float volume) {
        this.fuel = fuel;
        this.volume = volume;
    }

    @Override
    public String toString() {
        return "Engine{" + "fuel=" + fuel + ", volume=" + volume + '}';
    }

}

In [15]:
class Car implements Serializable {

    private static final long serialVersionUID = 2018L;

    private String name;
    private int year;
    private Color color;
    private transient Engine engine;

    public void setName(String name) {
        this.name = name;
    }

    public void setYear(int year) {
        this.year = year;
    }

    public void setColor(Color color) {
        this.color = color;
    }

    public void setEngine(Engine engine) {
        this.engine = engine;
    }

    @Override
    public String toString() {
        return "Car{" + "name=" + name + ", year=" + year + ", color=" + color + ", engine=" + engine + '}';
    }

}

Set some data

In [16]:
Car c = new Car();
c.setName("Fiat");
c.setYear(1998);
c.setColor(Color.BLUE);
c.setEngine(new Engine(Fuel.PETROL, 1.3f));

In [17]:
c

Car{name=Fiat, year=1998, color=BLUE, engine=Engine{fuel=PETROL, volume=1.3}}

In [18]:
import java.lang.reflect.Field;
Field idf = Car.class.getDeclaredField("serialVersionUID");
idf.setAccessible(true);

In [19]:
idf.get(c.getClass());

2018

In [20]:
writeToFile("out.dat",c)

In [21]:
Car c1 = (Car) readFromFile("out.dat")

In [22]:
c1 

Car{name=Fiat, year=1998, color=BLUE, engine=null}